In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

# Import from your codebase
from CLAPWrapper import CLAPWrapper
from datasets.esc50 import ESC50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [2]:
print("\n" + "="*80)
print("Loading ESC50 Dataset")
print("="*80)

root_path = "../data"
dataset = ESC50(root=root_path, download=False)
print(f"✅ Dataset loaded: {len(dataset)} samples")
print(f"   Classes: {len(dataset.classes)} categories")
print(f"   Sample classes: {dataset.classes[:5]}")

# Prepare text prompts
prompt = 'this is the sound of '
text_labels = [prompt + x for x in dataset.classes]
print(f"\n📝 Text prompts: {len(text_labels)} classes")


Loading ESC50 Dataset
Loading audio files


0it [00:00, ?it/s]

2000it [00:00, 14151.16it/s]

✅ Dataset loaded: 2000 samples
   Classes: 50 categories
   Sample classes: ['airplane', 'breathing', 'brushing teeth', 'can opening', 'car horn']

📝 Text prompts: 50 classes


In [12]:
# ============================================================================
# STEP 3: Initialize Models
# ============================================================================
print("\n" + "="*80)
print("Initializing Models")
print("="*80)

# Residual config con pc_weights = 1.0 (identità)
residual_config = {
    'mode': 'layer',
    'n_components_ratio': 1.0,
    'reweight_factor': 0.0,
    'target_layers': [0, 1, 2, 3],  # Layers dove applicare reweighting
}

print("\n🔧 Loading CLAP Standard...")
clap_standard = CLAPWrapper(
    version='2023',  # or '2022'
    use_cuda=torch.cuda.is_available(),
    type='classic'
)

print("\n🔧 Loading ResiDualCLAP...")
clap_residual = CLAPWrapper(
    version='2023',
    use_cuda=torch.cuda.is_available(),
    type='residual',
    residual_config=residual_config
)


Initializing Models

🔧 Loading CLAP Standard...

🔧 Loading ResiDualCLAP...

🔧 Setup ResiDual HTSAT
Modalità: LAYER
Target layers: [0, 1, 2, 3]
PCA components ratio: 1.0
Reweight factor: 0.0

✓ layer_0:
  Modalità: LAYER reweighting
  Layer dim: 96D → 96 PCs

✓ layer_1:
  Modalità: LAYER reweighting
  Layer dim: 192D → 192 PCs

✓ layer_2:
  Modalità: LAYER reweighting
  Layer dim: 384D → 384 PCs

✓ layer_3:
  Modalità: LAYER reweighting
  Layer dim: 768D → 768 PCs




In [13]:
# Prepare audio samples for PCA fitting
print("Collecting samples for PCA fitting (max 200 samples)...", end='')

# Create a simple dataloader wrapper per PCA fitting
class SimpleAudioDataset:
    def __init__(self, wrapper, esc50_dataset, max_samples=1000):
        self.wrapper = wrapper
        self.audio_paths = []
        for i in range(min(max_samples, len(esc50_dataset))):
            audio_path, _, _ = esc50_dataset[i]
            self.audio_paths.append(audio_path)
    
    def __len__(self):
        return len(self.audio_paths)
    
    def __getitem__(self, idx):
        audio_tensor = self.wrapper.load_audio_into_tensor(
            self.audio_paths[idx],
            self.wrapper.args.duration,
            resample=True
        )
        # ✅ Assicurati sia 1D
        if audio_tensor.dim() > 1:
            audio_tensor = audio_tensor.squeeze()
        
        return audio_tensor

# Create dataset and loader
pca_dataset = SimpleAudioDataset(clap_residual, dataset, max_samples=50)
pca_loader = DataLoader(
    pca_dataset, 
    batch_size=16, 
    shuffle=False,
    num_workers=0,  # Start with 0 for debugging
    pin_memory=False
)
print("OK")

In [5]:
pca_dataset[0].shape

torch.Size([308700])

In [6]:
list(pca_loader)[1].shape

torch.Size([16, 308700])

In [7]:
len(dataset)

2000

In [14]:
print("\n" + "="*80)
print("Fitting PCA Components")
print("="*80)

# Fit PCA
print(f"Fitting PCA on {len(pca_dataset)} samples...")
variance_ratios = clap_residual.clap.audio_encoder.base.htsat.fit_spectral_layers(
    pca_loader,
    max_samples=50
)

print("\n📊 PCA Variance Ratios:")
for layer_name, ratios in variance_ratios.items():
    print(f"   {layer_name}: Top 5 components = {ratios[:5]}")


Fitting PCA Components
Fitting PCA on 50 samples...

📊 Fitting Spectral Layers
Modalità: LAYER
Target layers: [0, 1, 2, 3]
Max samples: 50

📦 Fase 1: Raccolta hidden states...


Raccolta: 100%|██████████| 4/4 [00:10<00:00,  2.50s/it, samples=50]



✓ Raccolti 50 samples totali

🔬 Fase 2: Calcolo PCA e inizializzazione pesi...


✓ layer_0:
  Shape: torch.Size([204800, 96])
  Top-5 variance: ['0.2955', '0.0760', '0.0705', '0.0538', '0.0452']
  Cumulative top-5: 0.5412
  Total explained: 1.0000

✓ layer_1:
  Shape: torch.Size([51200, 192])
  Top-5 variance: ['0.1277', '0.0751', '0.0544', '0.0481', '0.0349']
  Cumulative top-5: 0.3403
  Total explained: 1.0000

✓ layer_2:
  Shape: torch.Size([12800, 384])
  Top-5 variance: ['0.0702', '0.0592', '0.0449', '0.0311', '0.0308']
  Cumulative top-5: 0.2362
  Total explained: 1.0000

✓ layer_3:
  Shape: torch.Size([3200, 768])
  Top-5 variance: ['0.1211', '0.0795', '0.0751', '0.0657', '0.0530']
  Cumulative top-5: 0.3944
  Total explained: 1.0000

✅ Fitting completato!


📊 PCA Variance Ratios:
   layer_0: Top 5 components = [0.2955418  0.07602653 0.07051117 0.05384455 0.04524239]
   layer_1: Top 5 components = [0.12771684 0.07512135 0.05444162 0.0481239  0.034897  ]
   layer_2: Top 5 compon

In [15]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_standard.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_baseline, y_labels = [], []

for i in tqdm(range(test_size), desc="Baseline"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_standard.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_standard.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_baseline.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_baseline_array = np.concatenate(y_preds_baseline, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_baseline_array, axis=1)
)

print(f"\n✅ Baseline Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


Baseline:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Baseline Accuracy: 0.920 (92.0%)


In [16]:
# Get text embeddings ONCE for all classes
text_embeddings = clap_residual.get_text_embeddings(text_labels)
print(f"   Text embeddings shape: {text_embeddings.shape}")

# Test on subset first (use full dataset later)
test_size = 100  # Start with 200 samples for speed
print(f"\n📊 Testing on {test_size} samples...")

y_preds_residual, y_labels = [], []

for i in tqdm(range(test_size), desc="residual"):
    # Get audio file path and label
    audio_path, target, one_hot_target = dataset[-(i+1)]
    
    # Get audio embedding
    audio_embedding = clap_residual.get_audio_embeddings([audio_path], resample=True)
    
    # Compute similarity
    similarity = clap_residual.compute_similarity(audio_embedding, text_embeddings)
    
    # Get prediction
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    y_preds_residual.append(y_pred)
    y_labels.append(one_hot_target.detach().cpu().numpy())

# Calculate accuracy
y_labels_array = np.concatenate(y_labels, axis=0)
y_preds_residual_array = np.concatenate(y_preds_residual, axis=0)

baseline_acc = accuracy_score(
    np.argmax(y_labels_array, axis=1), 
    np.argmax(y_preds_residual_array, axis=1)
)

print(f"\n✅ Residual Accuracy: {baseline_acc:.3f} ({baseline_acc*100:.1f}%)")

   Text embeddings shape: torch.Size([50, 1024])

📊 Testing on 100 samples...


residual:   0%|          | 0/100 [00:00<?, ?it/s]


✅ Residual Accuracy: 0.920 (92.0%)


In [17]:
audio_path, target, one_hot_target = dataset[0]

In [18]:
clap_residual.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])

In [19]:
clap_standard.get_audio_embeddings([audio_path], resample=True)

tensor([[ 0.7623,  0.2343, -0.5101,  ...,  1.8940,  0.9414, -0.0119]])